In [1]:
import sys
sys.path.append('/scratch/pradap/python-work/enrique/')
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

Installed autotime.py. To use it, type:
  %load_ext autotime


In [2]:
import pandas as pd
import magellan as mg

time: 433 ms


In [3]:
wal = mg.read_csv('../magellan/datasets/books/walmart.csv', dtype={'isbn':pd.np.str, 'pages':pd.np.str, 'volume':pd.np.str, 'editionNum':pd.np.str},  
                      low_memory=False, key='id')

time: 1.39 s


In [4]:
bwk = mg.read_csv('../magellan/datasets/books/bowker.csv', dtype={'isbn':pd.np.str, 'pages':pd.np.str, 'volume':pd.np.str, 'editionNum':pd.np.str},  
                      low_memory=False, key='id')

time: 51.7 ms


In [5]:
ab = mg.AttrEquivalenceBlocker()
candset = ab.block_tables(bwk, wal, 'isbn', 'isbn', ['title', 'author'], ['title', 'author'])

time: 262 ms


In [6]:
feat_table = mg.get_features_for_blocking(bwk, wal)

time: 2.35 s


In [7]:
sample_cset = mg.sample_one_table(candset, 30)

time: 1.9 ms


In [8]:
mg.init_jvm()

True

time: 154 ms


In [9]:
label_cset = mg.label(sample_cset, "gold_label")

time: 30.2 s


In [10]:
fv = mg.extract_feat_vecs(label_cset, ['ltable.title', 'rtable.title', 'ltable.author', 'rtable.author'], feat_table, ['gold_label'])

time: 579 ms


In [11]:
nb = mg.NBMatcher()
dt = mg.DTMatcher()
rf = mg.RFMatcher()

time: 1.11 ms


In [12]:
m = mg.select_matcher([nb, dt, rf], x=fv[list(feat_table['feature_name'])], y=fv['gold_label'])

time: 65.3 ms


In [14]:
mc, stats = mg.selector_matcher_combiner([nb, dt, rf], ['majority'], x=fv[list(feat_table['feature_name'])], y=fv['gold_label'])

time: 174 ms


In [15]:
stats

,matcher,mean score,name
0,<magellan.matcher.nbmatcher.NBMatcher object a...,0.500000,NaiveBayes
1,<magellan.matcher.dtmatcher.DTMatcher object a...,0.933333,DecisionTree
2,<magellan.matcher.rfmatcher.RFMatcher object a...,0.733333,RandomForest
3,<magellan.matcher.ensemblematcher.EnsembleMatc...,0.566667,"NaiveBayes,DecisionTree"
4,<magellan.matcher.ensemblematcher.EnsembleMatc...,0.466667,"NaiveBayes,RandomForest"
5,<magellan.matcher.ensemblematcher.EnsembleMatc...,0.800000,"DecisionTree,RandomForest"
6,<magellan.matcher.ensemblematcher.EnsembleMatc...,0.900000,"NaiveBayes,DecisionTree,RandomForest"


time: 7.99 ms


In [16]:
c_prime = mg.extract_feat_vecs(candset, attrs_before=['ltable.title', 'rtable.title', 'ltable.author', 'rtable.author'],
                              feat_table=feat_table)

time: 1.04 s


In [17]:
mc.fit(x=fv[list(feat_table['feature_name'])], y=fv['gold_label'])

time: 2.67 ms


In [18]:
res = mc.predict(x=c_prime[list(feat_table['feature_name'])])

time: 3 ms


In [19]:
app = mc.predict(table=c_prime, exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'ltable.title', 'rtable.title', 'ltable.author', 'rtable.author'], 
                 target_attr='predicted_label', append=True)

time: 24.4 ms


In [21]:
v = wal['volume']

time: 650 µs


In [27]:
aa = [a for a in v]

time: 7.85 ms


In [31]:
import numpy as np

time: 581 µs


In [32]:
aa[0] is np.NaN

True

time: 1.32 ms


In [36]:
mg._current_tokenizers['qgm_3']('muru')

[u'mur', u'uru']

time: 1.61 ms
